Задание ДЗ №27 <br>

Обучить берт для задачи классификации. <br>
Взять любой набор данных (например idmb) <br>
Важно рассписать что именно вы делаете. <br>

План работы: <br>
Импортируем нужные модули <br>
Загружаем датасет <br>
Обучаем модель <br>
Тестируем на тестовой выборке <br>

1. Загрузка необходимых модулей 

In [1]:
from transformers import MobileBertConfig, MobileBertModel
import pandas as pd

2. Загрузка и визуальная оценка датасета на примере первых пяти и последних пяти элементов

In [2]:
# Load the dataset
data = pd.read_csv("IMDB Dataset.csv")

# Check the data structure
print(data.head())
print(data.tail())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
                                                  review sentiment
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative


3. Преобразование оценки (положительной и отрицательной) в числовые метки (единица и ноль), разбиение датасета на тренировочный и тестовый

In [3]:
from sklearn.model_selection import train_test_split

# Map sentiment labels to numerical values
data['sentiment'] = data['sentiment'].map({'positive': 1, 'negative': 0})

# Split the data
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['review'], data['sentiment'], test_size=0.2, random_state=42
)

4. Токенизация данных <br>
Это означает, что мы преобразуем каждый элемент текста в числовой токен, поскольку модель не может работать с текстом напрямую. <br>
Для этого мы загружаем токенизатор MobileBERT. <br>
У нас среди парметров токенизатора есть max_length в 512 токенов. То есть, отзывы с большим числом слов будут обрезаться. <br>
Параметр padding=True приводит к тому, что если отзыв короче, чем 512 элементов, оставшиеся до 512 пустые места текста заполняются нулевыми токенами. <br>
В результате получится, что каждый отзыв преобразуется в строку из 512 числовых токенов. <br>
Модель BERT рассчитана на работу с такими строками токенов. <br>

In [4]:
from transformers import AutoTokenizer

# Load the MobileBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/mobilebert-uncased")

# Tokenize the data
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=512)

5. Определяем пользовательский датасет. <br>
Создаем датасеты для тренировочного и тестового наборов, как уже проделывали раньше в предыдущей работе. <br>
В методе класса getitem преобразуем элементы исходного датасета в тензоры pytorch

In [5]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Create datasets
train_dataset = IMDbDataset(train_encodings, train_labels.tolist())
test_dataset = IMDbDataset(test_encodings, test_labels.tolist())

6. Загрузка предобученной модели BERT <br>
В параметрах стоит num_labels=2. Это означает, что к предобученной модели добавляется линейный слой с двумя выходами, соответствующими двум классам для текстовых отзывов.

In [6]:
from transformers import MobileBertForSequenceClassification

# Load the model
model = MobileBertForSequenceClassification.from_pretrained("google/mobilebert-uncased", num_labels=2)

Some weights of MobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


7. Определяем даталоадеры и параметры для модели. <br>
Оптимизатор AdamW. <br>
Шедьюлер для скорости обучения устанавливает линейное убывание скорости обучения после каждого батча. <br>

In [7]:
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_scheduler

# Create data loaders
batch_size = 20
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle = False)

NumEpochs = 5 # Number of epochs for calculation
# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = len(train_loader) * NumEpochs  # Assuming NumEpochs epochs
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Move model to GPU if available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

MobileBertForSequenceClassification(
  (mobilebert): MobileBertModel(
    (embeddings): MobileBertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (embedding_transformation): Linear(in_features=384, out_features=512, bias=True)
      (LayerNorm): NoNorm()
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): MobileBertEncoder(
      (layer): ModuleList(
        (0-23): 24 x MobileBertLayer(
          (attention): MobileBertAttention(
            (self): MobileBertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=512, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): MobileBertSelfOutput(
              (dense): Linear(in_fe

8. Обучение, точнее, файн-тьюнинг предобученной модели BERT для распознавания класса отзыва. <br>
В ходе обучения выводим progress bar

In [8]:
from tqdm import tqdm

model.train()
for epoch in range(NumEpochs):  # Number of epochs to be followed
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        # Update progress bar
        loop.set_description(f"Epoch {epoch}")
        loop.set_postfix(loss=loss.item())

Epoch 4: 100%|██████████| 2000/2000 [17:47<00:00,  1.87it/s, loss=0.436]


9. Оценка модели на тестовой выборке

In [9]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        predictions = torch.argmax(outputs.logits, dim=-1)
        correct += (predictions == batch['labels']).sum().item()
        total += batch['labels'].size(0)

accuracy = correct / total
print(f"Test Accuracy: {accuracy*100:.2f}%")

Test Accuracy: 80.70%


10. Сохранение модели в файл

In [10]:
model.save_pretrained("mobilebert-imdb")
tokenizer.save_pretrained("mobilebert-imdb")

('mobilebert-imdb\\tokenizer_config.json',
 'mobilebert-imdb\\special_tokens_map.json',
 'mobilebert-imdb\\vocab.txt',
 'mobilebert-imdb\\added_tokens.json',
 'mobilebert-imdb\\tokenizer.json')

Выводы: <br>
На датасете imbd была дообучена модель BERT для обработки текстовой информации, а именно, определения, положителен или отрицателен отзыв на кинофильм. <br>

На тестовой выборке была достигнута следующая точность классификации: <br>
При размере датасета 1000 экземпляров, три эпохи: 44.5% (то есть даже чуть ниже, чем при случайном угадывании или если всем экземплярам тупо приписать класс "1" или, наоборот, "0") <br>
При размере датасета 2000 экземпляров, три эпохи: 62.1% <br>
При размере датасета 2000 экземпляров, четыре эпохи: 62.1% <br>
При полном размере датасета 50 000 экземпляров и счете на процессоре через пару часов происходит крах кернела Python (надо попробовать посчитать на видеокартке) <br>

При расчете на GPU: <br>
На полном размере датасета 50000 экземпляров, три эпохи, batch size = 16 - почаем точность 79.29% на тестовой выборке. <br>
При расчете в течение 5 эпох, batch size = 20 - точность на тестовой выборке   80.7%.

---